In [8]:
from selenium import webdriver  #Tarayıcı otomasyonu için WebDriver API'si
from selenium.webdriver.chrome.service import Service  #Chrome WebDriver hizmeti için yapılandırma
from webdriver_manager.chrome import ChromeDriverManager  #ChromeDriver'ı otomatik indirip kurar

#LCP ölçümü yapılacak websitenin URL'si
url = "https://www.n11.com/"

#webdriver-manager kullanarak webdriver kurulumu
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

try:
    driver.get(url)
    
    driver.implicitly_wait(10) #Sayfanın tamamen yüklenmesini beklemek için sabit bir süre (10 saniye)

    #LCP ölçümü için JavaScript kodu
    lcp_script = """
        let largestContentfulPaint = 0;
        const observer = new PerformanceObserver((list) => {
            const entries = list.getEntries();
            entries.forEach((entry) => {
                if(entry.startTime > largestContentfulPaint){
                    largestContentfulPaint=entry.startTime;
                }
            });
        });
        observer.observe({type: 'largest-contentful-paint', buffered: true});

        document.addEventListener('visibilitychange', () => {
            if (document.visibilityState === 'hidden'){
                observer.disconnect();
            }
        });

        // DOM yüklenmesi beklenir
        window.onload = () => {
            setTimeout(() => {
                observer.disconnect();
            }, 5000); //5 saniye sonra observer durdurulur
        };
        return largestContentfulPaint;
    """
    #JavaScript kodu çalıştırılarak LCP değeri alınır
    lcp_value = driver.execute_script(lcp_script)

    #LCP değerine göre sonucu İyi, Geliştirilebilir
    if lcp_value < 2500:  # 2.5 saniyeden küçükse "İyi" olarak kabul edilir
        result = "İyi"
    elif lcp_value < 4000:  # 2.5 ile 4 saniye arasındaysa "Geliştirilebilir" olarak kabul edilir
        result = "Geliştirilebilir"
    else:  # 4 saniyeden büyükse "Kötü" olarak kabul edilir
        result = "Kötü"

    # Sonuçları yazdır
    print(f"Site: {url}")
    print(f"LCP Değeri: {lcp_value:.2f} ms")  # LCP değerini milisaniye cinsinden yazdır
    print(f"LCP Sonucu: {result}")  # LCP sonucu

except Exception as e:
    # Eğer bir hata oluşursa hata mesajını yazdır
    print(f"Hata oluştu: {e}")

finally:
    # İşlem tamamlandığında driver'ı kapat
    driver.quit()


Site: https://www.n11.com/
LCP Değeri: 0.00 ms
LCP Sonucu: İyi
